In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pytest
from agent import BookingWorkflow
from dotenv import load_dotenv
load_dotenv()

workflow = BookingWorkflow()


In [5]:
# Test 1: Detect Intent for Making a Reservation
def test_detect_intent_make_reservation():
    state = {
        "user_message": "I want to book a room.",
        "not_filled_keys": workflow.NECESSARY_INFORMATION.copy(),
    }
    updated_state = workflow.run_graph(state)
    assert updated_state["intent"] == "make a reservation"
    print("\npass")


# Test 2: Collecting Information
def test_collect_information():
    state = {
        "user_message": "My name is John Doe, I need a room from 2024-10-20 to 2024-10-22.",
        "intent": "make a reservation",
        "not_filled_keys": workflow.NECESSARY_INFORMATION.copy(),
    }
    updated_state = workflow.run_graph(state)
    assert updated_state["full_name"] == "John Doe"
    assert updated_state["check_in_date"] == "2024-10-20"
    assert updated_state["check_out_date"] == "2024-10-22"
    assert "num_guests" in updated_state["not_filled_keys"]
    print("\npass")


# Test 3: Validation of Information (Valid Case)
def test_validate_information_valid():
    state = {
        "user_message": "I want to book a room for 2 people, and I'll pay by credit card.",
        "intent": "make a reservation",
        "full_name": "John Doe",
        "check_in_date": "2024-10-20",
        "check_out_date": "2024-10-22",
        "num_guests": 2,
        "payment_method": "credit card",
        "breakfast_included": True,
        "valid_info": None,
        "error": [],
        "not_filled_keys": [],
    }
    updated_state = workflow.validate_information(state)
    assert updated_state["valid_info"] == True
    assert updated_state["error"] == []
    print("\npass")


# Test 4: Validation of Information (Invalid Case)
def test_validate_information_invalid():
    state = {
        "user_message": "I'll be checking in on 2023-10-20 and checking out on 2023-10-18, but only for 0 guests.",
        "intent": "make a reservation",
        "full_name": "John Doe",
        "check_in_date": "2023-10-20",
        "check_out_date": "2023-10-18",
        "num_guests": 0,
        "payment_method": "credit card",
        "breakfast_included": True,
        "valid_info": None,
        "error": [],
        "not_filled_keys": [],
    }
    updated_state = workflow.validate_information(state)
    assert updated_state["valid_info"] == False
    assert "Check-in date cannot be in the past." in updated_state["error"]
    assert "Check-out date must be after check-in date." in updated_state["error"]
    assert "Number of guests must be positive." in updated_state["error"]
    print("\npass")


# Test 5: Generate Response
def test_generate_response():
    state = {
        "user_message": "I would like to book a room for 2 people from 2024-10-20 to 2024-10-22, with breakfast included.",
        "intent": "make a reservation",
        "full_name": "John Doe",
        "check_in_date": "2024-10-20",
        "check_out_date": "2024-10-22",
        "num_guests": 2,
        "payment_method": "credit card",
        "breakfast_included": True,
        "valid_info": True,
        "error": [],
        "not_filled_keys": [],
    }
    updated_state = workflow.generate_response(state)
    print(updated_state["response"])
    print("\npass")


# Test 6: Check Reservation Intent
def test_check_reservation():
    state = {
        "user_message": "Can you check my reservation?",
        "intent": None,
        "full_name": "John Doe",
        "check_in_date": "2024-10-20",
        "check_out_date": "2024-10-22",
        "num_guests": 2,
        "payment_method": "credit card",
        "breakfast_included": True,
        "valid_info": True,
        "error": [],
        "not_filled_keys": [],
    }
    updated_state = workflow.run_graph(state)
    print(updated_state["response"])
    assert updated_state["intent"] == "check reservation"
    print("\npass")


# Test 7: Changing the Reservation
def test_change_reservation():
    current_state = {
        "user_message": "On a second thought, I will not want breakfast. Besides, I prefer to pay with credit card. Change these",
        "intent": "make a reservation",
        "full_name": "Hugo Albuquerque Cosme da Silva",
        "check_in_date": "2024-10-23",
        "check_out_date": "2024-10-30",
        "num_guests": 3,
        "payment_method": "cash",
        "breakfast_included": True,
        "valid_info": True,
        "error": [],
        "not_filled_keys": [],
        "response": "Thank you, Hugo, for providing the details for your reservation. Here is a summary of your booking:\n\n- Check-in Date: 2024-10-23\n- Check-out Date: 2024-10-30\n- Number of Guests: 3\n- Payment Method: Cash\n- Breakfast Included: Yes\n\nWould you like to proceed with this booking?",
    }
    updated_state = workflow.run_graph(current_state)
    assert updated_state["breakfast_included"] == False
    assert updated_state["payment_method"] == "credit card"
    print("\npass")


In [6]:
# List of test functions to run
test_functions = [
    test_detect_intent_make_reservation,
    test_collect_information,
    test_validate_information_valid,
    test_validate_information_invalid,
    test_generate_response,
    test_check_reservation,
    test_change_reservation,
]

# Loop through each test function and run it
for test_function in test_functions:
    try:
        print(f"Running {test_function.__name__}...")
        test_function()
    except AssertionError as e:
        print(f"Test {test_function.__name__} failed: {e}")
    except Exception as e:
        print(f"An error occurred while running {test_function.__name__}: {e}")
    else:
        print(f"{test_function.__name__} passed.\n")


Running test_detect_intent_make_reservation...

pass
test_detect_intent_make_reservation passed.

Running test_collect_information...
next question:  Hello John! It looks like the check-in date you've selected is in the past. Could you please provide a correct check-in date? Additionally, it seems the number of guests hasn't been specified. Could you let us know how many guests will be staying? Thank you!

pass
test_collect_information passed.

Running test_validate_information_valid...
Test test_validate_information_valid failed: 
Running test_validate_information_invalid...

pass
test_validate_information_invalid passed.

Running test_generate_response...
Thank you, John, for providing the details for your reservation. Here's what I have so far:

- Check-in Date: 2024-10-20
- Check-out Date: 2024-10-22
- Number of Guests: 2
- Payment Method: Credit Card
- Breakfast Included: Yes

Could you please confirm if all the details are correct and if you would like to proceed with the booking